In [373]:
import pandas as pd
import numpy as np

In [374]:
df = pd.read_csv("BIKE DETAILS.csv")
df

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
0,Royal Enfield Classic 350,175000,2019,Individual,1st owner,350,NaN
1,Honda Dio,45000,2017,Individual,1st owner,5650,NaN
2,Royal Enfield Classic Gunmetal Grey,150000,2018,Individual,1st owner,12000,148114.0
3,Yamaha Fazer FI V 2.0 [2016-2018],65000,2015,Individual,1st owner,23000,89643.0
4,Yamaha SZ [2013-2014],20000,2011,Individual,2nd owner,21000,NaN
...,...,...,...,...,...,...,...
1056,Activa 3g,17000,2010,Individual,1st owner,500000,52000.0
1057,Honda CB twister,16000,2012,Individual,1st owner,33000,51000.0
1058,Bajaj Discover 125,15000,2013,Individual,2nd owner,35000,57000.0
1059,Honda CB Shine,12000,2009,Individual,1st owner,53000,58000.0


In [375]:
df.isna().sum()

name                   0
selling_price          0
year                   0
seller_type            0
owner                  0
km_driven              0
ex_showroom_price    435
dtype: int64

In [376]:
x = df.drop(labels="selling_price",axis=1)
x
y = df["selling_price"]
y

0       175000
1        45000
2       150000
3        65000
4        20000
         ...  
1056     17000
1057     16000
1058     15000
1059     12000
1060     10000
Name: selling_price, Length: 1061, dtype: int64

In [377]:
char_col = df.select_dtypes(include="object").columns
char_col

Index(['name', 'seller_type', 'owner'], dtype='object')

In [378]:
num_col = df.select_dtypes(include=np.number).columns.drop("selling_price")
num_col

Index(['year', 'km_driven', 'ex_showroom_price'], dtype='object')

In [379]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [380]:
#Build Numeric Pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler   
from sklearn.preprocessing import OneHotEncoder 
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


In [381]:
char_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ohn',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

In [382]:
from sklearn.compose import ColumnTransformer
preproccess = ColumnTransformer([
    ('num',numeric_pipeline,num_col),
    ('char',char_pipeline,char_col)
])

In [383]:
from sklearn.linear_model import LinearRegression
final_pipe = Pipeline([
    ('preproccess',preproccess),
    ('model',LinearRegression())
])

In [384]:
final_pipe.fit(x_train,y_train)

,steps,"[('preproccess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('char', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [385]:
y_pred = final_pipe.predict(x_test)

In [386]:
from sklearn.metrics import mean_absolute_percentage_error,r2_score
print(mean_absolute_percentage_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

0.4090535712960419
0.7882971649786962


In [387]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(final_pipe, x, y, cv=5)

In [389]:
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Cross-validation scores: [-0.04265049  0.22149413  0.69337856  0.31735505  0.64644556]
Mean cross-validation score: 0.36720456224301234
